In [6]:
import pandas as pd
import numpy as np
import re
from collections import defaultdict

In [2]:
aux_df = {}
for sheet in ['annual', 'quarterly', 'monthly']:
    aux_df[sheet] = pd.read_excel(
        r"D:\29 CS5228\03 Project\HDBResalePrice\data\auxiliary-data\macro_factors.xlsx", 
        sheet_name=sheet, engine='openpyxl')

In [9]:
df = pd.read_csv(r"D:\29 CS5228\03 Project\HDBResalePrice\outs\train_df_fe.csv")
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,eco_category,lease_commence_date,...,elevation,subzone,planning_area,region,resale_price,resale_year,resale_month,resale_quarter,flat_age,resale_price_sqm
0,2001-08,pasir ris,4 room,2,pasir ris drive 4,01 to 06,118.0,model a,uncategorized,1989,...,0.0,pasir ris drive,pasir ris,east region,209700.0,2001,8,3,12,1777.118644
1,2014-10,punggol,5 room,0,punggol field,10 to 15,110.0,improved,uncategorized,2003,...,0.0,punggol field,punggol,north-east region,402300.0,2014,10,4,11,3657.272727
2,2020-09,sengkang,5 room,2,fernvale lane,01 to 06,112.0,premium apartment,uncategorized,2004,...,0.0,fernvale,sengkang,north-east region,351000.0,2020,9,3,16,3133.928571
3,2000-10,clementi,3 room,0,clementi avenue 4,06 to 10,67.0,new generation,uncategorized,1980,...,0.0,clementi north,clementi,west region,151200.0,2000,10,4,20,2256.716418
4,2013-01,bukit batok,3 room,0,bukit batok street 11,06 to 10,73.0,model a,uncategorized,1985,...,0.0,bukit batok west,bukit batok,west region,318600.0,2013,1,1,28,4364.383562


In [8]:
df['resale_year'].unique()

array([2001, 2014, 2020, 2000, 2013, 2007, 2010, 2005, 2002, 2019, 2006,
       2009, 2015, 2004, 2008, 2012, 2011, 2003, 2018, 2017, 2016],
      dtype=int64)

In [4]:
def abbreviate_col_name(abrv_name):
    if ' ' in abrv_name:
        abrv_name = ''.join([s[0] if s[0].isupper() else (
            s if s.isnumeric() else '')for s in abrv_name.split(' ')])
        abrv_name = abrv_name.replace('_', '')
    return abrv_name

In [10]:
# annual
sheet='annual'
_aux_df = aux_df[sheet].copy()
# aux_df.pop(sheet)
orig_col_names = _aux_df.columns
new_col_names = [sheet+'_'+abbreviate_col_name(col) for col in orig_col_names]
_aux_df.columns = new_col_names
_aux_df = _aux_df.applymap(lambda x: np.nan if pd.isnull(x) else float(re.sub("[^\d\.]", "", str(x))))
df = pd.merge(df, _aux_df, how='left', left_on='resale_year', right_on=sheet+'_Variables')
df = df.drop(columns=sheet+'_Variables')
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,eco_category,lease_commence_date,...,resale_price_sqm,annual_TLF,annual_TEP,annual_TUP,annual_TURC,annual_RURC,annual_TP,annual_AMHIFWPHMECC,annual_NRFA,annual_NRFA
0,2001-08,pasir ris,4 room,2,pasir ris drive 4,01 to 06,118.0,model a,uncategorized,1989,...,1777.118644,2330.5,2267.3,63.2,2.7,3.8,4138012.0,1925.0,NaN,NaN
1,2014-10,punggol,5 room,0,punggol field,10 to 15,110.0,improved,uncategorized,2003,...,3657.272727,3530.8,3440.2,90.7,2.6,3.7,5469724.0,3418.0,4001.0,17673.0
2,2020-09,sengkang,5 room,2,fernvale lane,01 to 06,112.0,premium apartment,uncategorized,2004,...,3133.928571,3713.9,3574.0,139.9,3.8,5.2,5685807.0,4022.0,NaN,NaN
3,2000-10,clementi,3 room,0,clementi avenue 4,06 to 10,67.0,new generation,uncategorized,1980,...,2256.716418,2192.3,2094.8,97.5,4.4,6.0,4027887.0,1735.0,NaN,NaN
4,2013-01,bukit batok,3 room,0,bukit batok street 11,06 to 10,73.0,model a,uncategorized,1985,...,4364.383562,3443.7,3352.9,90.7,2.6,3.9,5399162.0,3204.0,4612.0,17552.0


In [11]:
# quarterly
df['resale_quarter'] = df['resale_month'].apply(lambda m: (m-1)//3 + 1)
sheet='quarterly'
_aux_df = aux_df[sheet].copy()
# aux_df.pop(sheet)
merge_on = ['resale_year', 'resale_quarter']
_aux_df[merge_on] = _aux_df['Variables'].str.split(' ', 1, expand=True)
_aux_df['resale_quarter'] = _aux_df['resale_quarter'].apply(lambda q: q[0])
_aux_df = _aux_df.drop(columns='Variables')
orig_col_names = _aux_df.columns
new_col_names = [sheet+'_'+abbreviate_col_name(col) if col not in merge_on else col for col in orig_col_names]
for col in merge_on:
    _aux_df[col] = _aux_df[col].astype(int)
    df[col] = df[col].astype(int)
_aux_df.columns = new_col_names
_aux_df = _aux_df.applymap(lambda x: float(re.sub("[^\d\.]", "", str(x))))
df = pd.merge(df, _aux_df, how='left', on=merge_on)
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,eco_category,lease_commence_date,...,annual_TUP,annual_TURC,annual_RURC,annual_TP,annual_AMHIFWPHMECC,annual_NRFA,annual_NRFA,quarterly_GICD,quarterly_ULCOOE,quarterly_HPI
0,2001-08,pasir ris,4 room,2,pasir ris drive 4,01 to 06,118.0,model a,uncategorized,1989,...,63.2,2.7,3.8,4138012.0,1925.0,NaN,NaN,47195.8,93.0,70.6
1,2014-10,punggol,5 room,0,punggol field,10 to 15,110.0,improved,uncategorized,2003,...,90.7,2.6,3.7,5469724.0,3418.0,4001.0,17673.0,105970.4,101.5,137.0
2,2020-09,sengkang,5 room,2,fernvale lane,01 to 06,112.0,premium apartment,uncategorized,2004,...,139.9,3.8,5.2,5685807.0,4022.0,NaN,NaN,113497.3,88.2,133.9
3,2000-10,clementi,3 room,0,clementi avenue 4,06 to 10,67.0,new generation,uncategorized,1980,...,97.5,4.4,6.0,4027887.0,1735.0,NaN,NaN,50041.6,99.2,75.8
4,2013-01,bukit batok,3 room,0,bukit batok street 11,06 to 10,73.0,model a,uncategorized,1985,...,90.7,2.6,3.9,5399162.0,3204.0,4612.0,17552.0,95412.4,101.0,148.6


In [12]:
import re
from time import strptime

# monthly
sheet='monthly'
_aux_df = aux_df[sheet].copy()
# aux_df.pop(sheet)
merge_on = ['resale_year', 'resale_month']
_aux_df[merge_on] = _aux_df['Variables'].str.split(' ', 1, expand=True)
_aux_df['resale_month'] = _aux_df['resale_month'].apply(lambda m: strptime(m,'%b').tm_mon)
_aux_df = _aux_df.drop(columns='Variables')
orig_col_names = _aux_df.columns
new_col_names = [sheet+'_'+abbreviate_col_name(col) if col not in merge_on else col for col in orig_col_names]
for col in merge_on:
    _aux_df[col] = _aux_df[col].astype(int)
    df[col] = df[col].astype(int)
_aux_df.columns = new_col_names
_aux_df = _aux_df.applymap(lambda x: float(re.sub("[^\d\.]", "", str(x))))
df = pd.merge(df, _aux_df, how='left', on=merge_on)
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,eco_category,lease_commence_date,...,annual_TP,annual_AMHIFWPHMECC,annual_NRFA,annual_NRFA,quarterly_GICD,quarterly_ULCOOE,quarterly_HPI,monthly_PLR,monthly_SDR,monthly_CPI
0,2001-08,pasir ris,4 room,2,pasir ris drive 4,01 to 06,118.0,model a,uncategorized,1989,...,4138012.0,1925.0,NaN,NaN,47195.8,93.0,70.6,5.80,1.28,75.299
1,2014-10,punggol,5 room,0,punggol field,10 to 15,110.0,improved,uncategorized,2003,...,5469724.0,3418.0,4001.0,17673.0,105970.4,101.5,137.0,5.35,0.11,99.255
2,2020-09,sengkang,5 room,2,fernvale lane,01 to 06,112.0,premium apartment,uncategorized,2004,...,5685807.0,4022.0,NaN,NaN,113497.3,88.2,133.9,5.25,0.10,100.139
3,2000-10,clementi,3 room,0,clementi avenue 4,06 to 10,67.0,new generation,uncategorized,1980,...,4027887.0,1735.0,NaN,NaN,50041.6,99.2,75.8,5.80,1.28,74.844
4,2013-01,bukit batok,3 room,0,bukit batok street 11,06 to 10,73.0,model a,uncategorized,1985,...,5399162.0,3204.0,4612.0,17552.0,95412.4,101.0,148.6,5.38,0.11,98.121


In [13]:
df.columns

Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'eco_category', 'lease_commence_date',
       'latitude', 'longitude', 'elevation', 'subzone', 'planning_area',
       'region', 'resale_price', 'resale_year', 'resale_month',
       'resale_quarter', 'flat_age', 'resale_price_sqm', 'annual_TLF',
       'annual_TEP', 'annual_TUP', 'annual_TURC', 'annual_RURC', 'annual_TP',
       'annual_AMHIFWPHMECC', 'annual_NRFA', 'annual_NRFA', 'quarterly_GICD',
       'quarterly_ULCOOE', 'quarterly_HPI', 'monthly_PLR', 'monthly_SDR',
       'monthly_CPI'],
      dtype='object')